In [1]:
import mysql.connector as connector
from mysql.connector import Error

from utils import get_default_connection_config, get_default_database, print_environment_info, print_rows, \
    call_procedure

try:
    connection = connector.connect(**get_default_connection_config(database=get_default_database()))
    print_environment_info(connection)
    cursor = connection.cursor()
except Error as e:
    print(f'Error: {e.errno}: {e.msg}')

Connector version: 8.0.32
Server version: 8.0.32


In [2]:
# Task 1
cursor.execute("DROP PROCEDURE IF EXISTS GetMaxQuantity")
cursor.execute("""
CREATE PROCEDURE GetMaxQuantity()
SELECT
    MAX(Quantity) AS "Max Quantity in Order"
FROM Orders
""")
call_procedure('GetMaxQuantity', cursor)

Max Quantity in Order
4


In [3]:
# Task 2
cursor.execute("""
PREPARE GetOrderDetail FROM 'SELECT OrderID, Quantity, TotalCost as Cost FROM Orders WHERE CustomerID = ?';
""")
cursor.execute("SET @id = 1")
query = """
EXECUTE GetOrderDetail USING @id
"""
print_rows(title='Prepared statement usage',
           query=query,
           cursor=cursor)

Prepared statement usage
------------------------
OrderID - Quantity - Cost
7 - 3 - 295.56
12 - 2 - 25.94
13 - 4 - 51.88


In [4]:
# Task 3
cursor.execute("DROP PROCEDURE IF EXISTS CancelOrder;")
cursor.execute("""CREATE PROCEDURE CancelOrder(IN cancelOrderId INT UNSIGNED)
DELETE FROM Orders WHERE OrderId = cancelOrderId;
""")

order_id_to_cancel = 1
print_rows(title=f'Before removing order with id {order_id_to_cancel}',
           query='SELECT COUNT(OrderID) FROM Orders WHERE OrderID = 1',
           cursor=cursor)
cursor.callproc('CancelOrder', args=[1])
print_rows(title=f'After removing order with id {order_id_to_cancel}',
           query='SELECT COUNT(OrderID) FROM Orders WHERE OrderID = 1',
           cursor=cursor)

Before removing order with id 1
-------------------------------
COUNT(OrderID)
1
After removing order with id 1
------------------------------
COUNT(OrderID)
0


In [5]:
assert cursor.close()
assert connection.close() is None